In [0]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import io
from sklearn.metrics import roc_auc_score
import xgboost as xgb 
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
# Seed the random number generator:
np.random.seed(1)

To upload the csv files

In [0]:

from google.colab import files
uploaded = files.upload()



MessageError: ignored

In [0]:
X = pd.read_csv("/content/train.csv")
Y = pd.read_csv("/content/test.csv")

In [0]:
train_label = X.iloc[:, -1]
test_label = Y.iloc[:, -1]

In [0]:
train_data = X.fillna(X.mean())
train_id = X["id"]
train_input = train_data.drop("id", axis = 1)
train_input['bid1_tot'] = train_input['bid1'] * train_input['bid1vol']
train_input['bid2_tot'] = train_input['bid2'] * train_input['bid2vol']
train_input['bid3_tot'] = train_input['bid3'] * train_input['bid3vol']
train_input['bid4_tot'] = train_input['bid4'] * train_input['bid4vol']
train_input['bid5_tot'] = train_input['bid5'] * train_input['bid5vol']

train_input['ask1_tot'] = train_input['ask1'] * train_input['ask1vol']
train_input['ask2_tot'] = train_input['ask2'] * train_input['ask2vol']
train_input['ask3_tot'] = train_input['ask3'] * train_input['ask3vol']
train_input['ask4_tot'] = train_input['ask4'] * train_input['ask4vol']
train_input['ask5_tot'] = train_input['ask5'] * train_input['ask5vol']

train_input['price_change'] = train_input['mid'] - train_input['last_price']

new_df = train_input.iloc[:, [i for i in range(0, 6)] + [i for i in range(-11, 0)]]

In [0]:
test_data = Y.fillna(Y.mean())
test_id = Y["id"]
test_input = test_data.drop("id", axis = 1)

test_input['bid1_tot'] = test_input['bid1'] * test_input['bid1vol']
test_input['bid2_tot'] = test_input['bid2'] * test_input['bid2vol']
test_input['bid3_tot'] = test_input['bid3'] * test_input['bid3vol']
test_input['bid4_tot'] = test_input['bid4'] * test_input['bid4vol']
test_input['bid5_tot'] = test_input['bid5'] * test_input['bid5vol']

test_input['ask1_tot'] = test_input['ask1'] * test_input['ask1vol']
test_input['ask2_tot'] = test_input['ask2'] * test_input['ask2vol']
test_input['ask3_tot'] = test_input['ask3'] * test_input['ask3vol']
test_input['ask4_tot'] = test_input['ask4'] * test_input['ask4vol']
test_input['ask5_tot'] = test_input['ask5'] * test_input['ask5vol']

test_input['price_change'] = test_input['mid'] - test_input['last_price']

test_edited_df = test_input.iloc[:, [i for i in range(0, 6)] + [i for i in range(-11, 0)]]

In [0]:
model = xgb.XGBRegressor(n_estimators = 100, max_depth = 3, learning_rate = 0.1, n_jobs = 4)
model.fit(new_df, train_label, eval_metric = 'auc')


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[22:17:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
importance = model.feature_importances_


/usr/local/lib/python3.6/dist-packages/xgboost/sklearn.py:546: RuntimeWarning: invalid value encountered in true_divide
  return all_features / all_features.sum()


In [0]:
new_df

,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1_tot,bid2_tot,bid3_tot,bid4_tot,bid5_tot,ask1_tot,ask2_tot,ask3_tot,ask4_tot,ask5_tot,price_change
0,3842.4,3842.6,1.330144,1.989035,103.0,0,30739.200000,3842.000000,23050.800000,53774.000000,23042.400000,23056.800000,3843.400000,3843.600000,38438.000000,7688.000000,0.2
1,3842.8,3843.4,6.000000,49.000000,55.0,-43,26901.000000,23056.800000,42266.400000,3842.000000,23050.800000,3843.800000,15376.000000,15377.600000,3844.600000,49982.400000,0.6
2,3844.0,3844.3,7.000000,77.000000,84.0,-69,11531.400000,3843.600000,15372.800000,80703.000000,46113.600000,3844.800000,61520.000000,38454.000000,15382.400000,34612.200000,0.3
3,3843.8,3843.4,3.000000,34.000000,37.0,-30,38430.000000,49956.400000,46108.800000,7684.000000,15365.600000,7687.600000,26908.000000,3844.400000,7689.200000,42292.800000,-0.4
4,3843.2,3843.1,3.000000,38.000000,41.0,-35,53799.200000,46108.800000,7684.000000,7683.600000,15365.600000,3843.400000,11532.000000,3844.600000,42292.800000,57675.000000,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411429,4024.0,4024.2,1.330144,1.989035,0.0,0,28168.000000,20119.000000,8047.200000,16092.800000,36207.000000,8048.800000,12073.800000,20124.000000,24150.000000,8050.400000,0.2
411430,4024.0,4024.2,1.330144,1.989035,0.0,0,28168.000000,20119.000000,8047.200000,16092.800000,36207.000000,8048.800000,12073.800000,20124.000000,24150.000000,8050.400000,0.2
411431,4024.0,4024.2,1.330144,1.989035,0.0,0,28168.000000,20119.000000,8047.200000,16092.800000,36207.000000,8048.800000,12073.800000,20124.000000,24150.000000,8050.400000,0.2
411432,4024.4,4024.2,0.000000,1.000000,1.0,0,24144.000000,20119.000000,8047.200000,16092.800000,36207.000000,4024.400000,12073.800000,20124.000000,24150.000000,8050.400000,-0.2


In [0]:
predictions = pd.DataFrame({'id': np.array(test_id), 'Predicted': model.predict(test_edited_df)}, columns=['id', 'Predicted'])
predictions.to_csv("/content/predictions_XGBR_100_3_fixedbid1.csv", index = False)

In [0]:
model = xgb.XGBRegressor(n_estimators = 200, max_depth = 4, learning_rate = 0.1, n_jobs = 4)
num_folds = 5
kf = KFold(n_splits=num_folds)
test_preds = []
for train_index, test_index in kf.split(new_df):
    # Training and testing data points for this fold:
    x_train, x_test = new_df.iloc[train_index], new_df.iloc[test_index]
    y_train, y_test = train_label.iloc[train_index], train_label.iloc[test_index]
    model.fit(x_train, y_train, eval_metric = 'auc')
    test_preds.append(model.predict(test_edited_df))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:00:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored

In [0]:
np.mean(test_preds, axis = 0)

array([0.60777926, 0.25202194, 0.2528187 , ..., 0.49297348, 0.37555057,
       0.4307537 ], dtype=float32)

In [0]:
def modelfit(alg, dtrain, predictors,useTrainCV=False, cv_folds=2, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain, label=train_label)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], train_label,eval_metric='auc')
        
    #Predict training set:
    dtrain_predprob = alg.predict(dtrain[predictors])
        
    #Print model report:
    print ("\nModel Report")
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(train_label, dtrain_predprob))
    return alg.predict(test_edited_df)

In [0]:
predictors = [x for x in new_df]
xgb1 = xgb.XGBRegressor(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
predictions = modelfit(xgb1, new_df, predictors)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \



Model Report
AUC Score (Train): 0.668092


In [0]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBRegressor( learning_rate =0.1, n_estimators=100, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', n_jobs = -1, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=3)
gsearch1.fit(new_df,train_label)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=-1, nthread=None,
                                    objective='binary:logistic', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=27, silent=None,
                                    subsample=0.8, verbosity=1),
             iid=False, n_jobs=4,
             param_grid={'max_depth': range(3, 10, 2),
                         'min_child_weight': range(1, 6, 2)},
             pre_

In [0]:
predictions = pd.DataFrame({'id': np.array(test_id), 'Predicted': gsearch1.predict(test_edited_df)}, columns=['id', 'Predicted'])
predictions.to_csv("/content/predictions_XGBR_gridsearch.csv", index = False)

In [0]:
train = pd.read_csv("/content/norm_train_input.csv")
test = pd.read_csv("/content/norm_test_input.csv")

In [0]:
new_df = train.iloc[:, :-1]
new_df = new_df.drop("id", axis = 1)
test_df = test.iloc[:, :-1]
test_df = test_df.drop("id", axis = 1)


In [0]:
test_df

,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1tot,ask1tot,bid2tot,ask2tot,bid3tot,ask3tot,bid4tot,ask4tot,bid5tot,ask5tot
0,0.520535,0.520992,0.017857,0.113872,0.086957,0.500000,0.017042,0.000216,0.125594,0.028647,0.016899,0.000195,0.000177,0.035748,0.000180,0.009078
1,0.303725,0.304389,0.017857,0.000000,0.007246,0.598214,0.000105,0.030647,0.008391,0.065911,0.008392,0.000114,0.000104,0.000108,0.075952,0.000108
2,0.515759,0.519561,0.035714,0.000000,0.014493,0.607143,0.059188,0.061804,0.142282,0.028650,0.000175,0.000196,0.000175,0.009077,0.042676,0.035752
3,0.115568,0.118321,0.000000,0.020704,0.014493,0.580357,0.058093,0.000050,0.032935,0.000046,0.024711,0.091594,0.016484,0.043775,0.000040,0.008788
4,0.572111,0.572996,0.000000,0.000000,0.000000,0.589286,0.000196,0.020810,0.033707,0.009726,0.000194,0.000217,0.016949,0.018028,0.017236,0.026940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,0.387775,0.389790,0.000000,0.000000,0.000000,0.589286,0.000134,0.000162,0.033398,0.019015,0.016765,0.000146,0.025080,0.017827,0.008592,0.008984
191855,0.254059,0.255725,0.017857,0.020704,0.021739,0.580357,0.016770,0.152410,0.000087,0.150186,0.033172,0.027712,0.000087,0.000090,0.159883,0.132021
191856,0.589303,0.587786,0.000000,0.000000,0.000000,0.589286,0.000201,0.000244,0.008582,0.009734,0.000199,0.000221,0.000198,0.009125,0.000201,0.000209
191857,0.394460,0.395038,0.071429,0.020704,0.043478,0.607143,0.025302,0.061433,0.033408,0.000151,0.041726,0.009406,0.058360,0.000140,0.304643,0.000140


In [0]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBRegressor( learning_rate =0.1, n_estimators=100, max_depth=3,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', n_jobs = -1, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=3)
gsearch1.fit(new_df,train_label)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=-1, nthread=None,
                                    objective='binary:logistic', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=27, silent=None,
                                    subsample=0.8, verbosity=1),
             iid=False, n_jobs=4,
             param_grid={'max_depth': range(3, 10, 2),
                         'min_child_weight': range(1, 6, 2)},
             pre_

In [0]:
preds = gsearch1.predict(test_df)

In [0]:
preds

array([0.5284759 , 0.21810246, 0.3164651 , ..., 0.3868499 , 0.41881248,
       0.5804453 ], dtype=float32)

In [0]:
predictions = pd.DataFrame({'id': np.array(test_id), 'Predicted': preds}, columns=['id', 'Predicted'])
predictions.to_csv("/content/predictions_XGBR_gridsearch1.csv", index = False)

In [0]:
predictions

,id,Predicted
0,592380,0.528476
1,592381,0.218102
2,592382,0.316465
3,592383,0.619517
4,592384,0.221810
...,...,...
191854,784234,0.382271
191855,784235,0.249798
191856,784236,0.386850
191857,784237,0.418812


In [0]:
from sklearn.ensemble import BaggingRegressor

In [0]:
model = xgb.XGBClassifier(n_estimators = 100, max_depth = 3, learning_rate = 0.1, n_jobs = -1)
clf = BaggingClassifier(base_estimator=model,
                        n_estimators=10, random_state=0).fit(new_df,train_label)

In [0]:
preds = clf.predict_proba(test_edited_df)[:, 1]
predictions = pd.DataFrame({'id': np.array(test_id), 'Predicted': preds}, columns=['id', 'Predicted'])
predictions.to_csv("/content/predictions_XGBR_baggingclass.csv", index = False)

In [0]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5)

In [0]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.tree import DecisionTreeRegressor


In [0]:
DTC = DecisionTreeRegressor()

ada_best = AdaBoostRegressor(DTC, random_state=7, n_estimators = 10,
                            learning_rate = 0.1)

ada_best.fit(new_df,train_label)


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                       criterion='mse',
                                                       max_depth=None,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       presort='deprecated',
                                                       random_state=None,
                                                       splitter='best'),
               

In [0]:
xgb_best = xgb.XGBRegressor(n_estimators = 100, max_depth = 3, learning_rate = 0.1, n_jobs = -1)
xgb_best.fit(new_df,train_label)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[08:26:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [0]:
max(ada_best.predict_proba(test_edited_df)[:, 1])

0.9999999999999998

In [0]:
GBC_best = GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.1,
                                 max_depth = 3, min_samples_leaf = 20)
GBC_best.fit(new_df,train_label)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=20, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
votingC = VotingRegressor(estimators=[('adac',ada_best),('xgb',xgb_best), ('gbc',GBC_best)], n_jobs=-1)

votingC = votingC.fit(new_df,train_label)

In [0]:
preds = votingC.predict(test_edited_df)


In [0]:
preds

array([0.40030958, 0.50520091, 0.15906132, ..., 0.65835549, 0.25100412,
       0.27860403])

In [0]:
predictions = pd.DataFrame({'id': np.array(test_id), 'Predicted': preds}, columns=['id', 'Predicted'])
predictions.to_csv("/content/predictions_votingregress.csv", index = False)

In [0]:
new_df

,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1_tot,bid2_tot,bid3_tot,bid4_tot,bid5_tot,ask1_tot,ask2_tot,ask3_tot,ask4_tot,ask5_tot,price_change
0,3842.4,3842.6,1.400583,1.964277,103.0,0,30739.2,3842.0,23050.8,53774.0,23042.4,23056.8,3843.4,3843.6,38438.0,7688.0,0.2
1,3842.8,3843.4,6.000000,49.000000,55.0,-43,26901.0,23056.8,42266.4,3842.0,23050.8,3843.8,15376.0,15377.6,3844.6,49982.4,0.6
2,3844.0,3844.3,7.000000,77.000000,84.0,-69,11531.4,3843.6,15372.8,80703.0,46113.6,3844.8,61520.0,38454.0,15382.4,34612.2,0.3
3,3843.8,3843.4,3.000000,34.000000,37.0,-30,38430.0,49956.4,46108.8,7684.0,15365.6,7687.6,26908.0,3844.4,7689.2,42292.8,-0.4
4,3843.2,3843.1,3.000000,38.000000,41.0,-35,53799.2,46108.8,7684.0,7683.6,15365.6,3843.4,11532.0,3844.6,42292.8,57675.0,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592375,4110.2,4110.3,1.000000,1.000000,2.0,1,8220.4,8220.0,4109.4,12327.6,8218.0,8220.8,12335.4,41120.0,28785.4,28788.2,0.1
592376,4109.4,4110.5,6.000000,5.000000,11.0,1,12327.6,8218.0,8217.2,24649.2,20540.0,12335.4,37008.0,28785.4,28788.2,20565.0,1.1
592377,4109.4,4110.5,1.400583,1.964277,0.0,0,12327.6,8218.0,8217.2,24649.2,20540.0,12335.4,37008.0,28785.4,28788.2,20565.0,1.1
592378,4109.4,4110.5,1.400583,1.964277,0.0,0,12327.6,8218.0,8217.2,24649.2,20540.0,12335.4,37008.0,28785.4,28788.2,20565.0,1.1


In [0]:
test_edited_df

,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1_tot,bid2_tot,bid3_tot,bid4_tot,bid5_tot,ask1_tot,ask2_tot,ask3_tot,ask4_tot,ask5_tot,price_change
0,5178.4,5178.3,1.000000,11.000000,12.0,-10,15534.6,82848.0,15533.4,5177.6,5177.4,5178.4,20714.4,5178.8,25895.0,10359.6,-1.000000e-01
1,5133.0,5132.9,1.000000,0.000000,1.0,1,5132.8,10265.2,10264.8,5132.2,51320.0,20532.0,41065.6,5133.8,5134.0,5134.2,-1.000000e-01
2,5177.4,5178.0,2.000000,0.000000,2.0,2,41417.6,93182.4,5176.6,5176.4,31056.0,36251.6,20716.0,5179.2,10358.8,25898.0,6.000000e-01
3,5093.6,5093.9,0.000000,2.000000,2.0,-1,40745.6,25465.0,20371.2,15276.6,5092.0,5094.6,5094.8,56045.0,30571.2,10190.8,3.000000e-01
4,5189.2,5189.2,1.444629,1.936579,0.0,0,5188.8,25943.0,5188.4,15564.6,15564.0,15568.8,10381.2,5190.8,15573.0,20764.8,1.818989e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,5150.6,5150.8,1.444629,1.936579,0.0,0,5150.6,25752.0,15450.6,20600.0,10299.6,5151.0,15454.2,5151.6,15456.0,10304.8,2.000000e-01
191855,5122.6,5122.7,1.000000,2.000000,3.0,-1,15367.8,5122.4,25611.0,5121.8,102432.0,81964.8,87091.0,20492.8,5123.4,81977.6,1.000000e-01
191856,5192.8,5192.3,1.444629,1.936579,0.0,0,5192.0,10383.6,5191.2,5190.4,5190.2,5192.6,10385.6,5193.0,10386.4,5193.4,-5.000000e-01
191857,5152.0,5151.9,4.000000,2.000000,6.0,2,20607.2,25758.0,30908.4,41209.6,190587.0,36064.0,5152.2,10304.8,5152.6,5152.8,-1.000000e-01
